In [ ]:
!pip install requests

I adapted [this notebook](https://github.com/Elucidation/Ngram-Tutorial/blob/master/NgramTutorial.ipynb) to Python 3. 

## IPython Notebook - N-gram Tutorial

Here is the explanation from the original author:

_What we want to do is build up a dictionary of N-grams, which are pairs, triplets or more (the N) of words that pop up in the training data, with the value being the number of times they showed up. After we have this dictionary, as a naive example we could actually predict sentences by just randomly choosing words within this dictionary and doing a weighted random sample of the connected words that are part of n-grams within the keys._

_Lets see how far we can get with N-grams without outside resources._

_We have a text file for [Pride and Prejudice from Project Gutenberg](https://www.gutenberg.org/ebooks/1342) stored as pg1342.txt in the same folder as our notebook, but also available online directly. Let's load the text to a string since it's only 701KB, which will fit in memory nowadays._

_**Note** : If we wanted to be more memory efficient we should parse the text file on a line or character by character basis, storing as needed, etc._

In [116]:
# Find the number links by looking on Project Gutenberg in the address bar for a book.
books = {'Pride and Prejudice': '1342',
         'Huckleberry Fin': '76',
         'Sherlock Holmes': '1661'}

book = books['Pride and Prejudice']

# Load text from Project Gutenberg URL
import requests
url_template = 'https://www.gutenberg.org/cache/epub/%s/pg%s.txt'

response = requests.get(url_template % (book, book), 'r')
txt = response.text

# See the number of characters and the first 50 characters to confirm it is there    
print(len(txt), ',', txt[:50] , '...')

717572 , ﻿The Project Gutenberg EBook of Pride and Prejudic ...


Great, now lets split into words into a big list, splitting on anything non-alphanumeric [A-Za-z0-9] (as well as punctuation) and forcing everything lowercase:

In [117]:
import re
words = re.split('[^A-Za-z]+', txt.lower())
words = list(filter(None, words)) # Remove empty strings

# Print length of list
print(len(words))

125897


## N-grams generation
From this we can now generate N-grams, lets start with a 1-gram, basically the set of all the words

**Note** : One could use a dictionary instead of a set and keeping count of the occurances gives word frequency

In [118]:
# Create set of all unique words, this throws away any information about frequency however
gram1 = set(words)

print(len(gram1))

# Print 20 elements of the set only
print(list(gram1)[:20])

6528
['withstood', 'burnt', 'spanish', 'commissioned', 'implies', 'cents', 'figures', 'family', 'guilt', 'camp', 'ventured', 'shades', 'effect', 'aught', 'form', 'conjecture', 'sides', 'noblest', 'independence', 'wavering']


Lets try and get the 2-gram now, which is pairs of words. Let's have a quick look to see the last 10 and how they look.

In [119]:
# See the last 10 pairs
for i in range(len(words)-10, len(words)-1):
    print(words[i], words[i+1])

subscribe to
to our
our email
email newsletter
newsletter to
to hear
hear about
about new
new ebooks


Okay, seems good, lets get all word pairs, and then generate a set of unique pairs from it

In [120]:
word_pairs = [(words[i], words[i+1]) for i in range(len(words)-1)]
print(len(word_pairs))

gram2 = set(word_pairs)
print(len(gram2))

# Print 20 elements from gram2
print(list(gram2)[:20])

125896
55636
[('had', 'often'), ('i', 'get'), ('gone', 'mr'), ('sit', 'by'), ('married', 'at'), ('any', 'public'), ('his', 'reason'), ('if', 'a'), ('her', 'story'), ('not', 'yours'), ('must', 'marry'), ('its', 'writer'), ('attention', 'which'), ('any', 'disrespect'), ('and', 'decisions'), ('precipitance', 'which'), ('ladyship', 'received'), ('has', 'the'), ('disposition', 'was'), ('upon', 'my')]


## N-Grams Frequency
Okay, that was fun, but this isn't enough, we need frequency if we want to have any sense of probabilities, which is what N-grams are about. Instead of using sets, lets create a dictionary with counts

In [121]:
gram1 = dict()

# Populate 1-gram dictionary
for word in words:
    if word in gram1:
        gram1[word] += 1
    else:
        gram1[word] = 1 # Start a new entry with 1 count since saw it for the first time.

# Turn into a list of (word, count) sorted by count from most to least
gram1 = sorted(gram1.items(), key=lambda item: -item[1])

# Print top 20 most frequent words
print([(word, freq) for word, freq in gram1[:20]])

[('the', 4507), ('to', 4243), ('of', 3730), ('and', 3658), ('her', 2225), ('i', 2070), ('a', 2012), ('in', 1937), ('was', 1847), ('she', 1710), ('that', 1594), ('it', 1550), ('not', 1450), ('you', 1428), ('he', 1339), ('his', 1271), ('be', 1260), ('as', 1192), ('had', 1177), ('with', 1100)]


For Pride and Prejudice, the words 'the', 'to', 'of', and 'and' were the top four most common words. Sounds about right, not too interesting yet, lets see what happens with 2-grams.

In [122]:
gram2 = dict()

# Populate 2-gram dictionary
for i in range(len(words)-1):
    key = (words[i], words[i+1])
    if key in gram2:
        gram2[key] += 1
    else:
        gram2[key] = 1

# Turn into a list of (word, count) sorted by count from most to least
gram2 = sorted(gram2.items(), key=lambda item: -item[1])

# Print top 20 most frequent words
print([(word, freq) for word, freq in gram2[:20]])

[(('of', 'the'), 491), (('to', 'be'), 445), (('in', 'the'), 397), (('i', 'am'), 303), (('mr', 'darcy'), 273), (('to', 'the'), 268), (('of', 'her'), 261), (('it', 'was'), 251), (('of', 'his'), 235), (('she', 'was'), 212), (('she', 'had'), 205), (('had', 'been'), 200), (('it', 'is'), 194), (('i', 'have'), 188), (('to', 'her'), 179), (('that', 'he'), 177), (('could', 'not'), 167), (('he', 'had'), 166), (('and', 'the'), 165), (('for', 'the'), 163)]


It looks like "of the" and "to be" are the top two most common 2-grams, sounds good.

## Next word prediction

What can we do with this? Well lets see what happens if we take a random word from all the words, and build a sentence by just choosing the most common pair that has that word as it's start.

In [123]:
start_word = words[int(len(words)/4)]
print(start_word)

enough


I just went ahead and chose the word that appears $1/4$ of the way into words, random enough.

Now in a loop, iterate through the frequency list (most frequent first) and see if it matches the first word in a pair, if so, the next word is the second element in the word pair, and continue with that word. Stop after N words or the list does not contain that word.

**Note** : gram2 is a list that contains (key,value) where key is a word pair (first, second),
           so you need element[0][0] for first word and element [0][1] for second word

In [124]:
def get2GramSentence(word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Find Next word
        word = next((element[0][1] for element in gram2 if element[0][0] == word), None)
        if not word:
            break
    return ' '.join(words)

word = start_word
print("Start word:", word)

print('2-gram sentence: "', get2GramSentence(word, 20), '"')

Start word: enough
2-gram sentence: " enough to be so much as to be so much as to be so much as to be so much "


It gets stuck in a loop pretty much straight away. Not very interesting, try out other words and see what happens.

In [125]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print("Start word:", word)
    print('2-gram sentence: "', get2GramSentence(word, 20), '"')

Start word: and
2-gram sentence: " and the whole party were to be so much as to be so much as to be so much as "
Start word: he
2-gram sentence: " he had been so much as to be so much as to be so much as to be so much "
Start word: she
2-gram sentence: " she was not be so much as to be so much as to be so much as to be so "
Start word: when
2-gram sentence: " when she was not be so much as to be so much as to be so much as to be "
Start word: john
2-gram sentence: " john with the whole party were to be so much as to be so much as to be so much "
Start word: never
2-gram sentence: " never be so much as to be so much as to be so much as to be so much as "
Start word: i
2-gram sentence: " i am sure i am sure i am sure i am sure i am sure i am sure i am "
Start word: how
2-gram sentence: " how much as to be so much as to be so much as to be so much as to be "


## Weighted random choice based on frequency

**This is our simple probabilistic MLE N-gram model**

Same thing. Okay, lets randomly choose from the subset of all 2grams that matches the first word, using a weighted-probability based on counts.

In [126]:
import random
def weighted_choice(choices):
    total = sum(w for c, w in choices)
    r = random.uniform(0, total)
    upto = 0
    for c, w in choices:
        if upto + w > r:
            return c
        upto += w
    
def get2GramSentenceRandom(word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram2 if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]
    return ' '.join(words)

In [127]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print("Start word:", word)
    print('2-gram sentence: "', get2GramSentenceRandom(word, 20), '"')

Start word: and
2-gram sentence: " and jane mr collins has happened but you please her own and did sir such kinds of its contents as "
Start word: he
2-gram sentence: " he came only hope the days longer she had the company and then with no fresh source of chance in "
Start word: she
2-gram sentence: " she can you have just been a cheerful humour and good jokes did the hill my father did not unlikely "
Start word: when
2-gram sentence: " when i have suffered such behaviour in which they may incur by speaking to you could be kept them every "
Start word: john
2-gram sentence: " john with grave propriety of my dear eliza glancing over every thing and knowledge of civility to communicate and are "
Start word: never
2-gram sentence: " never in the easiness of shame and concluded had the gentleman and substitute a moment been to see your daughter "
Start word: i
2-gram sentence: " i believe us be likely to walk and lydia has neither at the attics are wasting your father cannot say "
Start wo

Now that's way more interesting! Those are starting to look like sentences!

Let's try a longer sentence

In [128]:
word = 'it'
print("Start word:", word)
print('2-gram sentence: "', get2GramSentenceRandom(word, 50), '"')

Start word: it
2-gram sentence: " it rapid and pray tell the very wisely resolved to spend a disappointment at all wish at rosings certainly bestowed on her friends an airing would probably been a carriage as much better that you did not exactly defined she dreaded to know we can i can this is it "


Pretty cool, lets see what happens when we go to N-grams above 2.

## Tri-grams and more
Okay, let's create a Ngram generator that can let us make ngrams of arbitrary sizes

In [129]:
def generateNgram(inputwords,n=1):
    gram = dict()
    
    # Some helpers to keep us crashing the PC for now
    assert n > 0 and n < 100
    
    # Populate N-gram dictionary
    for i in range(len(inputwords)-(n-1)):
        key = tuple(inputwords[i:i+n])
        if key in gram:
            gram[key] += 1
        else:
            gram[key] = 1

    # Turn into a list of (word, count) sorted by count from most to least
    gram = sorted(gram.items(), key=lambda item: -item[1])
    return gram

trigram = generateNgram(words,3)
# Print top 20 most frequent ngrams
print(trigram[:20])

[(('i', 'do', 'not'), 62), (('i', 'am', 'sure'), 62), (('project', 'gutenberg', 'tm'), 57), (('as', 'soon', 'as'), 55), (('she', 'could', 'not'), 50), (('that', 'he', 'had'), 37), (('it', 'would', 'be'), 34), (('in', 'the', 'world'), 34), (('i', 'am', 'not'), 32), (('the', 'project', 'gutenberg'), 31), (('i', 'dare', 'say'), 31), (('it', 'was', 'not'), 30), (('could', 'not', 'be'), 30), (('that', 'he', 'was'), 29), (('mr', 'darcy', 's'), 29), (('that', 'it', 'was'), 28), (('on', 'the', 'subject'), 28), (('of', 'mr', 'darcy'), 27), (('would', 'have', 'been'), 27), (('as', 'well', 'as'), 27)]


In [298]:
#Added inputwords as first parameter in order to used in in part2
def getNGramSentenceRandom(gram, word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]
    return ' '.join(words)

for n in range(2,10):
    # Generate ngram list
    print(f"Generating {n}-gram list...")
    ngram = generateNgram(words,n)
    print("Done")
    
    # Try out a bunch of sentences
    for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how','country']:
        print("Start word:", word)
        print(f'{n}-gram sentence: \n"{getNGramSentenceRandom(ngram, word, 20)}"')
        print()
        
    print('***************************************************************************')

Generating 2-gram list...
Done
Start word: and
2-gram sentence: 
"and coming with she else dearly and illustrious good with that and have the had therefore i sorrow object speaking"

Start word: he
2-gram sentence: 
"he their test considering to the to i represented see and is which your say make away courage a parasol"

Start word: she
2-gram sentence: 
"she had forget resolved she if at would not many you miss will of no of so far thought guard"

Start word: when
2-gram sentence: 
"when so than there capable all him tm lucas it more to were welcomed unlink make but it whose left"

Start word: john
2-gram sentence: 
"john had ungenerous any discovered employed plead them way know there never i been natural up manners possible and for"

Start word: never
2-gram sentence: 
"never the after had extravagance speak which tete yourself had humiliating now it i return it but have a family"

Start word: i
2-gram sentence: 
"i could the had were gone by might surprise from that know or elizabe

# Continue Assignment

# Part 1

In [268]:
import math
import random
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from operator import itemgetter
import numpy as np

In [269]:
def countNGram(frequency):
    count ={}
    
    for (key,fvalue) in reversed(sorted(frequency.items(), key = itemgetter(1))):
        count[key] = fvalue
 
    return count

In [270]:
def getNGramList(count_dict):
    l = count_dict 
    l=list(l.values())
    l.sort()

    d={}
    for i in range(len(l)):
        count =1
        for j in range(i,0,-1):
            if l[i] == l[j-1]:
                count = count + 1
            else:
                break 
    
        d[l[i]] = count
    return (d)


In [271]:
class goodTuring:
    def __init__(self,Ncount,Ucount): #dictionary of unigram and 4gram count
        self.nGramCount = Ncount.copy()
        self.UniGramCount = Ucount.copy()
        
        self.ndictN = getNGramList(self.nGramCount)
        self.UnidictN = getNGramList(self.UniGramCount)
        
        self.keysn,self.valuesn = list(self.ndictN.keys()),list(self.ndictN.values())    
        self.keysuni,self.valuesuni = list(self.UnidictN.keys()),list(self.UnidictN.values()) 
        
        self.NewCountN()
        self.NewCountuni()

    def NewCountN(self):
               
        for i in self.nGramCount.keys():
            try:
                self.nGramCount[i] = (self.nGramCount[i]+1)*(self.ndictN[self.nGramCount[i]+1]/self.ndictN[self.nGramCount[i]])
            except:
                intr= np.interp(float(self.nGramCount[i]),[float(j) for j in self.keysn],[float(k) for k in self.valuesn])

                self.nGramCount[i] = (self.nGramCount[i]+1)*(intr/(self.ndictN[self.nGramCount[i]]))

        return self.nGramCount 

    def NewCountuni(self):
               
        for i in self.UniGramCount.keys():
            try:
                self.UniGramCount[i] = (self.UniGramCount[i]+1)*(self.UnidictN[self.UniGramCount[i]+1]/self.UnidictN[self.UniGramCount[i]])
            except:
                intr= np.interp(float(self.UniGramCount[i]),[float(j) for j in self.keysuni],[float(k) for k in self.valuesuni])

                self.UniGramCount[i] = (self.UniGramCount[i]+1)*(intr/(self.UnidictN[self.UniGramCount[i]]))


        return self.UniGramCount 
    
    def Probability(self,a,b,c,d, tokenNgram): 
        ngram = d +" "+ c +" "+ b +" "+ a
        if ngram not in self.nGramCount.keys():
            return (self.ndictN[1]/len(tokenNgram))
        
        return self.nGramCount[ngram]/self.UniGramCount[b]


In [272]:
def Perplexity(data,bigramLst,model):
    log_perplxty = 0
    for i in data:
        log_prob = 0
        a = generateNgram([i],4)
        for i in a:
            print(i)
            new = model.Probability(i.split(" ")[3],i.split(" ")[2],i.split(" ")[1],i.split(" ")[0],bigramLst)
            log_prob = log_prob + math.log(new)
            
        log_perplxty = log_perplxty+log_prob
        log_perplxty = (-1/len(data))*(log_perplxty)
       
    return (math.exp(log_perplxty) )

In [273]:
#Split words into train-test
random.shuffle(words)

train = words[:int(len(words)*0.8)]#80%
test= words[int(len(words)*0.8):]#20%

In [274]:
unigramLst = generateNgram(train,1)
quadgramLst= generateNgram(train,4)
fr   =  nltk.FreqDist(quadgramLst)
feq  =  nltk.FreqDist(unigramLst)
count_unigram = countNGram(feq)
count_quadgram = countNGram(fr)

In [275]:
#Split text into sentences using nltk
sentences = nltk.sent_tokenize(txt)

In [276]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
sentences_clean = [p for p in sentences if p not in stop_words]

In [277]:
#Create the language models on the training corpus (Fourgram)

from nltk.util import ngrams
unigram=[]
fourgram=[]
tokenized_text = []
for sentence in sentences_clean:
    sentence = sentence.lower()
    sequence = word_tokenize(sentence) 
    for word in sequence:
        if (word =='.'):
            sequence.remove(word) 
        else:
            unigram.append(word)
    tokenized_text.append(sequence) 
    fourgram.extend(list(ngrams(sequence, 4)))

def removal(x):     
    y = []
    for pair in x:
        count = 0
        for word in pair:
            if word in stop_words:
                count = count or 0
            else:
                count = count or 1
        if (count==1):
            y.append(pair)
    return(y)        
fourgram = removal(fourgram)
freq_four = nltk.FreqDist(fourgram)
print("Most common n-grams without stopword removal and without add-1 smoothing: \n")
print ("\nMost common fourgrams: ", freq_four.most_common(5))

Most common n-grams without stopword removal and without add-1 smoothing: 


Most common fourgrams:  [((',', "''", 'said', 'she'), 41), ((',', "''", 'said', 'elizabeth'), 37), (("''", 'said', 'she', ','), 34), ((',', 'i', 'am', 'sure'), 29), ((',', "''", 'said', 'he'), 28)]


In [278]:
#Create dictionary where each element is a list corresponding to a particular n-gram, and store every word and its associated probability as elements of the list. Add-1 smoothing is performed considering all the unique words in the tokenized text of our dataset as the vocabulary.

#Add-1 smoothing is performed here.
            
ngrams_all = {1:[], 2:[], 3:[], 4:[]}
for i in range(4):
    for each in tokenized_text:
        for j in ngrams(each, i+1):
            ngrams_all[i+1].append(j);
ngrams_voc = {1:set([]), 2:set([]), 3:set([]), 4:set([])}
for i in range(4):
    for gram in ngrams_all[i+1]:
        if gram not in ngrams_voc[i+1]:
            ngrams_voc[i+1].add(gram)
total_ngrams = {1:-1, 2:-1, 3:-1, 4:-1}
total_voc = {1:-1, 2:-1, 3:-1, 4:-1}
for i in range(4):
    total_ngrams[i+1] = len(ngrams_all[i+1])
    total_voc[i+1] = len(ngrams_voc[i+1])                       
    
ngrams_prob = {1:[], 2:[], 3:[], 4:[]}
for i in range(4):
    for ngram in ngrams_voc[i+1]:
        tlist = [ngram]
        tlist.append(ngrams_all[i+1].count(ngram))
        ngrams_prob[i+1].append(tlist)
    
for i in range(4):
    for ngram in ngrams_prob[i+1]:
        ngram[-1] = (ngram[-1]+1)/(total_ngrams[i+1]+total_voc[i+1])             #add-1 smoothing

In [279]:
#Prints top 10 unigram, bigram, trigram, fourgram after smoothing
print("Most common n-grams without stopword removal and with add-1 smoothing: \n")
for i in range(4):
    ngrams_prob[i+1] = sorted(ngrams_prob[i+1], key = lambda x:x[1], reverse = True)
    
print ("\nMost common fourgrams: ", str(ngrams_prob[4][:20]))

Most common n-grams without stopword removal and with add-1 smoothing: 


Most common fourgrams:  [[(',', "''", 'said', 'she'), 0.0001755082426192516], [(',', "''", 'said', 'elizabeth'), 0.00015879317189360857], [("''", 'said', 'she', ','), 0.0001462568688493763], [(',', 'i', 'am', 'sure'), 0.00012536303044232257], [(',', "''", 'said', 'he'), 0.00012118426276091181], [('said', 'she', ',', '``'), 0.00011282672739809031], [("''", 'said', 'he', ','), 0.0001044691920352688], [(',', 'my', 'dear', ','), 9.193288899103655e-05], [("''", 'said', 'elizabeth', ','), 8.77541213096258e-05], [(',', 'she', 'could', 'not'), 8.357535362821503e-05], [('i', 'do', 'not', 'know'), 8.357535362821503e-05], [('*', '*', '*', '*'), 8.357535362821503e-05], [(',', "''", 'replied', 'elizabeth'), 7.939658594680428e-05], [(',', "''", 'said', 'her'), 7.939658594680428e-05], [('said', 'he', ',', '``'), 7.521781826539353e-05], [(',', "''", 'said', 'mrs.'), 7.521781826539353e-05], [(',', 'i', 'suppose', ','), 7.10390505

In [280]:
#Execute using good-turing
probTuring = goodTuring(count_quadgram,count_unigram)

# Part 2

In [281]:
#Select Sherlock Holmes book and get it directly from url
book = books['Sherlock Holmes']

# Load text from Project Sherlock Holmes URL
import requests
url_template = 'https://www.gutenberg.org/cache/epub/%s/pg%s.txt'

response = requests.get(url_template % (book, book), 'r')
txt2 = response.text

print(len(txt2), ',', txt2[:75] , '...')

594916 , ﻿Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doy ...


In [282]:
#Get words of selected text
import re
words_sh = re.split('[^A-Za-z]+', txt2.lower())
words_sh = list(filter(None, words_sh)) 

print(len(words_sh))

109000


In [283]:
#Shuffle words and split to train and test
random.shuffle(words_sh)

train_sh = words_sh[:int(len(words_sh)*0.8)]#80%
test_sh= words_sh[int(len(words_sh)*0.8):]#20%

In [284]:
len(test_sh)

21800

In [285]:
fourGram = generateNgram(train_sh,4)

In [297]:
for n in range(4,5):
    # Generate ngram list
    print(f"Generating {n}-gram list...")
    ngram = generateNgram(words_sh,n)
    print("Done")
    
    # Try out some of sentences
    for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how','country']:
        print("Start word:", word)
        print(f'{n}-gram sentence: \n"{getNGramSentenceRandom(ngram, word, 20)}"')
        print()
        
    print('***************************************************************************')

Generating 4-gram list...
Done
Start word: and
4-gram sentence: 
"and white the my which facts man of order do glance recent your it the the had that and you"

Start word: he
4-gram sentence: 
"he proud seemed books see would his incident there the for me i seeing up as in some lady thief"

Start word: she
4-gram sentence: 
"she to swaying yes that now father with pounds even america your it old impatient on for press until horse"

Start word: when
4-gram sentence: 
"when that heard occupy america your you exacted man the made room idea explained i in for point departure confederates"

Start word: john
4-gram sentence: 
"john passed governesses to merest of lock no centre park by a you any but for a o brass you"

Start word: never
4-gram sentence: 
"never a was i paper cleaver the or about lad conditions to roof it written said hair had until rearranging"

Start word: i
4-gram sentence: 
"i clearly you singularly at have do the yes flicking gentle she acquirement dissatisfied i because

In [303]:
#list containing n-gram
quadgramLst= generateNgram(train_sh,4)
print (quadgramLst[:10])
#  query for count of unigram
fr   =  nltk.FreqDist(quadgramLst)

count_4gram = countNGram(fr)

[(('and', 'and', 'the', 'i'), 2), (('the', 'a', 'and', 'been'), 2), (('to', 'the', 'he', 'would'), 2), (('a', 'had', 'the', 'and'), 2), (('the', 'it', 'which', 'the'), 2), (('of', 'to', 'the', 'a'), 2), (('to', 'i', 'from', 'was'), 2), (('the', 'no', 'that', 'and'), 2), (('it', 'the', 'the', 'i'), 2), (('i', 'and', 'the', 'the'), 2)]


In [289]:
probTuring = goodTuring(count_4gram,count_unigram)

In [290]:
#Generate NGram model for test set
fourGramLst = generateNgram(test,4)

In [295]:
fourGramLst[:5]

[(('her', 'in', 'very', 'on'), 2),
 (('to', 'was', 'ask', 'per'), 1),
 (('was', 'ask', 'per', 'jenkinson'), 1),
 (('ask', 'per', 'jenkinson', 'delivered'), 1),
 (('per', 'jenkinson', 'delivered', 'than'), 1)]

In [294]:
#Perplexity
per = Perplexity(fourGramLst,quadgramLst,probTuring)
print ("Good Turing Result:",per)

Good Turing Result: 1.0


In [293]:
#Good turing performs better than add one.